In [31]:
import polars as pl
from datetime import datetime, timedelta

pl.Config.set_fmt_str_lengths(200)
pl.Config.set_fmt_float("full")

polars.config.Config

Objective:

The objective is to identify potentially good preconfirmation examples and how long different types of transactions linger in the mempool on Holesky

In [2]:
# start and end date range
start_date = datetime(2024, 5, 30)
end_date = datetime(2024, 6, 1)


# list comprehension to generate endpoints
mempool_urls = [
    f"https://data.ethpandaops.io/xatu/holesky/databases/default/mempool_transaction/{
        date.year}/{date.month}/{date.day}.parquet"
    for date in (
        start_date + timedelta(days=n) for n in range((end_date - start_date).days + 1)
    )
]

# list comprehension to generate endpoints
beacon_urls = [
    f"https://data.ethpandaops.io/xatu/holesky/databases/default/canonical_beacon_block_execution_transaction/{
        date.year}/{date.month}/{date.day}.parquet"
    for date in (
        start_date + timedelta(days=n) for n in range((end_date - start_date).days + 1)
    )
]

In [7]:
# Create a list of Lazy DataFrames for each Parquet file
mempool_df = pl.concat(
    [pl.scan_parquet(file) for file in mempool_urls]
).group_by("hash").agg(
    [
        pl.col('event_date_time').min().alias('eaerliest_time_in_mempool')
    ]
)

In [8]:
beacon_df = pl.concat(
    [pl.scan_parquet(file) for file in beacon_urls]
)

In [11]:
txs_df = beacon_df.join(
    mempool_df, on="hash", how="inner", coalesce=True
).collect()

In [13]:
txs_df.shape

(1888395, 50)

In [28]:
txs_cleaned_df = (
    txs_df
    .select(
        'slot',
        'slot_start_date_time',
        'eaerliest_time_in_mempool',
        'hash',
        'from',
        'to',
        'type',
    )
    .with_columns(
        pl.from_epoch('slot_start_date_time', time_unit='s').dt.timestamp("ms")
    )
    .with_columns(
        ((pl.col('slot_start_date_time') - pl.col('eaerliest_time_in_mempool')
          ) / 1000).alias('seconds_in_mempool')
    )
    .select('slot', 'seconds_in_mempool', 'hash', 'from', 'to', 'type')
)

In [44]:
txs_cleaned_df.sort(by='type')

slot,seconds_in_mempool,hash,from,to,type
u32,f64,binary,binary,binary,u8
1775823,7.479,"b""0x000dac1b4b5862878a79b4d417235a3bedc166c5c3d7f9e3a8700d79b9c09dc1""","b""0x614d45cB18842C708899820bc6B284Fcd93D445A""","b""0xbb3fB2aD154E6fCa6DCD35082843d3a5819431c1""",0
1765186,4.671,"b""0xa9ddc7176add5c389f92a1ce2e351905b58072fde1c7af000bf6332a4701b690""","b""0xF7F72A1F5cdF5d2B6670aF5b60e0e0Ad22ef7Bce""","b""0xEA593b730d745FB5fE01B6d20e6603915252c6BF""",0
1766648,21.369,"b""0xdb7f213db333156678f7ee87d11ff0ccb6b57210a204f2676ac071fbf79107c6""","b""0x6CbBE2fcb7b2054a754e212f98Bd11e64Ee5CE4c""","b""0xEA593b730d745FB5fE01B6d20e6603915252c6BF""",0
1766081,2.418,"b""0x792e01067e8e44d0f09e7edbfd4a401672b0a3439a55913096c5216772a89d74""","b""0x625BCC1142E97796173104A6e817Ee46C593b3C5""","b""0x1c8A29F7517c7ceA04B98b80723D871a99DD3a2c""",0
1765123,23.425,"b""0x50bd3c863aeac0eb4f7ed6d97e98338b83fb9f3f18382005bea98ed863d00713""","b""0x898e4430A57Ec5b424dFfe06D8Aa0442519e1F59""","b""0xFE1355874C2537480fCCc5AB62D36B1bE5e7230b""",0
…,…,…,…,…,…
1762930,3109.732,"b""0xbc67c8529d41d1088d6281a945638954632fdc241c252182e28006fec9a20fe6""","b""0xa66008075589d4253c0fe26eD520e9ff88605f88""","b""0x79C9109b764609df928d16fC4a91e9081F7e87DB""",3
1763940,15234.199,"b""0x06611832eedc509fa0a6ff555d37f4f10639419325b84e03c9f6d012d3369c0b""","b""0xED3D68aCf5366785859A96e66a2778A02d14f08e""","b""0x79C9109b764609df928d16fC4a91e9081F7e87DB""",3
1761503,-0.49,"b""0xb4b9bd4195bd08a6906caf86a17b5195e38c5009ca65ea3be82a50917f571849""","b""0x8511a42aC38975b7c24b1Fa70bB2f344cd55DFB2""","b""0x79C9109b764609df928d16fC4a91e9081F7e87DB""",3


In [43]:
txs_cleaned_df.filter(
    pl.col('from') == '0x5CC894D9CB74B57AdDF2CB267d1df05463Abb12a'.lower())

slot,seconds_in_mempool,hash,from,to,type
u32,f64,binary,binary,binary,u8


In [41]:
# the to transactions are the addresses/mostly contracts that are interacted with the most on holesky.
txs_cleaned_df.group_by('to').agg(
    pl.col('seconds_in_mempool').mean().round(
        3).alias('average_seconds_in_mempool'),
    pl.len().alias('tx_count'),
    pl.col('type').mean().round(0).alias('type'),
    pl.col('seconds_in_mempool').min().round(
        3).alias('min_seconds_in_mempool'),
    pl.col('seconds_in_mempool').max().round(
        3).alias('max_seconds_in_mempool'),
).sort(by='average_seconds_in_mempool', descending=True).filter(pl.col('type') == 3)

to,average_seconds_in_mempool,tx_count,type,min_seconds_in_mempool,max_seconds_in_mempool
binary,f64,u32,f64,f64,f64
"b""0xd289D207C0AfD1588C402EC6EA4697e7Bd59ae28""",30555.424,1,3,30555.424,30555.424
"b""0x1f71C24b9115D5bf1A923dED05336FB7b6A7Bc95""",5146.07,7,3,8.989,25329.755
"b""0x84E9a8007DD9a95c76E0693622522D36e9911Abd""",669.864,25,3,4.882,11707.019
"b""0xaDD87ed6c64e7847C0a96Db859c9F6372D7996Db""",412.624,5,3,3.898,1452.061
"b""0xff00000000000000000000000000000082823100""",26.977,3,3,3.447,38.84
"b""0xBAc1000000000000000000000000000000000000""",12.396,122,3,0.243,176.316
"b""0xff00000000000000000000000000000000017920""",11.796,513,3,-0.894,122.456
"b""0xFf00000000000000000000000000000017000920""",9.972,12,3,-0.083,19.418


Analysis Conclusion:

There is not a lot of activity that could be good preconfirmations on Holesky such as defi swaps and blobs. Need to find a mempool data method to get more up to date holesky mempool data to get a better idea of current contract activity as the date lags behind by about a month (June 1st is latest data but today is June 23rd)